# Imports

In [1]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import models
import loader
import training
import metrics
import autotune

# Flags

In [2]:
TRAIN_MNIST = False
TRAIN_CIFAR = False

# GPU

In [3]:
device = torch.device("cuda")

# Data Loading

In [4]:
mnist_data_path = './train-images-idx3-ubyte'
mnist_label_path = './train-labels-idx1-ubyte'

cifar_data_path = './cifar-10-batches-py'

In [5]:
#5-fold generation
mnist_data = loader.MNIST(mnist_data_path, mnist_label_path, 5)
mnist_splits = mnist_data.splits

cifar_data = loader.CIFAR(cifar_data_path, 5)
cifar_splits = cifar_data.splits

In [6]:
print("MNIST STATS")
print("Number of splits:", len(mnist_splits))
print("Number of segments per split (train, val, test):", len(mnist_splits[0]))
print("Info per segment (data, labels):", len(mnist_splits[0][0]))
print("Size of segement (num examples):", len(mnist_splits[0][0][0]))
print()
print("CIFAR STATS")
print("Number of splits:", len(cifar_splits))
print("Number of segments per split (train, val, test):", len(cifar_splits[0]))
print("Info per segment (data, labels):", len(cifar_splits[0][0]))
print("Size of segement (num examples):", len(cifar_splits[0][0][0]))

MNIST STATS
Number of splits: 5
Number of segments per split (train, val, test): 3
Info per segment (data, labels): 2
Size of segement (num examples): 48000

CIFAR STATS
Number of splits: 5
Number of segments per split (train, val, test): 3
Info per segment (data, labels): 2
Size of segement (num examples): 40000


# Model & Optimizer

In [7]:
model = models.VanillaCNN
optimizer = optim.Adam
mnist_in_channels=(1,)
cifar_in_channels=(3,)

# Training and Evaluation

## MNIST

In [8]:
if TRAIN_MNIST:
    mnist_trainer = training.Trainer(model, optimizer, mnist_data, device)

In [9]:
if TRAIN_MNIST:
    mnist_trainer.train(mnist_in_channels, 1e-3, 2, 128, 8, verbose=False, num_loss=3)

In [10]:
if TRAIN_MNIST:
    mnist_trainer.test(mnist_in_channels, 128, 12)

In [11]:
if TRAIN_MNIST:
    mnist_plotter = metrics.Plotter(mnist_trainer.val_metrics)

In [12]:
if TRAIN_MNIST:
    mnist_plotter.plotLoss("Loss")

In [13]:
if TRAIN_MNIST:
    mnist_plotter.plotLoss("Loss", mnist_trainer.metrics())

In [14]:
if TRAIN_MNIST:
    mnist_plotter.plotAccuracy("Acc")

## CIFAR

In [15]:
if TRAIN_CIFAR:
    cifar_trainer = training.Trainer(model, optimizer, cifar_data, device)

In [16]:
if TRAIN_CIFAR:
    cifar_trainer.train(cifar_in_channels, 1e-3, 1, 32, 8, verbose=False, num_loss=3)

In [17]:
if TRAIN_CIFAR:
    cifar_trainer.test(cifar_in_channels, 128, 12)

# Hyperparam Search

In [18]:
from torch import nn
class TuneNN(nn.Module):
    def __init__(self, in_channels, hidden_dims):
        super(TuneNN, self).__init__()

        self.blockOne = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dims, 3, padding=1),
            nn.BatchNorm2d(hidden_dims),
            nn.ReLU(True),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)
        )
        
        self.blockTwo = nn.Sequential(
            nn.Conv2d(hidden_dims, hidden_dims, 3, padding=1),
            nn.BatchNorm2d(hidden_dims),
            nn.ReLU(True),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)
        )
        
        self.blockThree = nn.Sequential(
            nn.Conv2d(hidden_dims, hidden_dims, 3, padding=1),
            nn.BatchNorm2d(hidden_dims),
            nn.ReLU(True),
            nn.Dropout2d(0.1),
        )
        
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        self.linear = nn.Sequential(
            nn.Linear(hidden_dims, 10),
        )
            
    def forward(self, x):
        x = self.blockOne(x)
        x = self.blockTwo(x)
        x = self.blockThree(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [19]:
# model params - *args as tuple (model based)
# train params - *args as tuple (learning rate, epochs, batch size, number of workers)

config_1 = {
    'model' : (1,8),
    'train' : (1e-2, 1, 32, 8)
}

config_2 = {
    'model' : (1,32),
    'train' : (1e-3, 1, 32, 8)
}

config_3 = {
    'model' : (1,64),
    'train' : (1e-3, 1, 32, 8)
}

configs = [config_3, config_2, config_1]

In [20]:
hyperparam = autotune.Autotuner(TuneNN, optimizer, mnist_data, device)

In [21]:
hyperparam.search(configs)

[Fold 1] Epoch:1 Training Acc:0.76825
[Fold 1] Epoch:1 Validation Acc:0.965
[Fold 2] Epoch:1 Training Acc:0.7764583333333334
[Fold 2] Epoch:1 Validation Acc:0.9706666666666667
[Fold 3] Epoch:1 Training Acc:0.7766666666666666
[Fold 3] Epoch:1 Validation Acc:0.966
[Fold 4] Epoch:1 Training Acc:0.7803541666666667
[Fold 4] Epoch:1 Validation Acc:0.9681666666666666
[Fold 5] Epoch:1 Training Acc:0.7714583333333334
[Fold 5] Epoch:1 Validation Acc:0.9643333333333334
Best Fold Validation Results: [0.965   0.97067 0.966   0.96817 0.96433]
Finished Cross Validation Training
[Fold: 1] Testing Acc: 0.9638333333333333
[Fold: 2] Testing Acc: 0.9706666666666667
[Fold: 3] Testing Acc: 0.9673333333333334
[Fold: 4] Testing Acc: 0.969
[Fold: 5] Testing Acc: 0.9688333333333333
[Fold 1] Epoch:1 Training Acc:0.6501666666666667
[Fold 1] Epoch:1 Validation Acc:0.9378333333333333
[Fold 2] Epoch:1 Training Acc:0.648375
[Fold 2] Epoch:1 Validation Acc:0.9445
[Fold 3] Epoch:1 Training Acc:0.6355208333333333
[Fold 

In [22]:
hyperparam.bestConfig()

{'model': (1, 64), 'train': (0.001, 1, 32, 8), 'results': 0.9679333333333332}